In [1]:


import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import os

cwd = os.getcwd()
print(cwd)


C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


C:\Users\joonhay2\Desktop


In [2]:

############ FUNCTIONS ##############
def mapping_county_to_region(county_name):
    return county_production_df[county_production_df['County'] == county_names.upper()]["Ag District"]

def edit_price_df(price_data):
    ordered_price_data = pd.DataFrame(index = price_data.index, columns = price_data.columns)
    row_index = 0;
    price_data_yearly_grouped = price_data.groupby("Year")
    for years in price_data["Year"].drop_duplicates():
        temp_df = price_data_yearly_grouped.get_group(years)
        for i in range(len(temp_df)):
            ordered_price_data.iloc[row_index] = temp_df.iloc[len(temp_df)-1-i]
            row_index += 1

    #ordered_price_data.to_csv(path + '/IL_Corn_Price.csv')
    return ordered_price_data


In [4]:

############### MAIN #################
# Files
path = cwd

county_name_file = "\County_Name_IL_updated.csv"
county_production_df = pd.read_csv(path + county_name_file)

price_file = "\IL_Corn_Price.csv"
price_df = pd.read_csv(path + price_file)

weather_file = "\weather_data_updated.csv"
weather_df = pd.read_csv(path + weather_file)


# Weighted Average.
region_names = county_production_df["Ag District"].drop_duplicates()
grouped_df = county_production_df.groupby(["Ag District"])
weather_group_df = weather_df.groupby(["COUNTY"])
total_production = county_production_df["Value"].sum()
county_weight_df = pd.DataFrame(index=county_production_df["County"].drop_duplicates(), columns=["Weight"])

for i in region_names:
    print(grouped_df.get_group(i))

    
    



      State Ag District  County Commodity     Value
0  ILLINOIS     CENTRAL  MCLEAN      CORN  63008000
1  ILLINOIS     CENTRAL   LOGAN      CORN  41005000
2  ILLINOIS     CENTRAL   MACON      CORN  30430000
      State Ag District      County Commodity     Value
3  ILLINOIS        EAST    IROQUOIS      CORN  62773000
4  ILLINOIS        EAST  LIVINGSTON      CORN  54632000
5  ILLINOIS        EAST   CHAMPAIGN      CORN  47650000
      State     Ag District   County Commodity     Value
6  ILLINOIS  EAST SOUTHEAST   SHELBY      CORN  32652000
7  ILLINOIS  EAST SOUTHEAST    EDGAR      CORN  32018000
8  ILLINOIS  EAST SOUTHEAST  DOUGLAS      CORN  22507000
       State Ag District    County Commodity     Value
9   ILLINOIS   NORTHEAST  LA SALLE      CORN  58757000
10  ILLINOIS   NORTHEAST   DE KALB      CORN  46132000
11  ILLINOIS   NORTHEAST    GRUNDY      CORN  21406000
       State Ag District  County Commodity     Value
12  ILLINOIS   NORTHWEST  BUREAU      CORN  53809000
13  ILLINOIS  

In [33]:

for county in county_weight_df.index:
    county_weight_df.loc[county] = (county_production_df[county_production_df["County"] == county]["Value"] / total_production).iloc[0]

for county in county_production_df["County"]:
    weather_group_df.get_group(county.lower())

for county in weather_df["COUNTY"]:
    weather_group_df.get_group(county.lower())

year_month_columns = weather_group_df.get_group(weather_df["COUNTY"].iloc[0])[["YEAR", "MONTH"]]

In [6]:
county_weight_df

,Weight
County,
MCLEAN,0.0655619
LOGAN,0.0426671
MACON,0.0316634
IROQUOIS,0.0653174
LIVINGSTON,0.0568464
CHAMPAIGN,0.0495814
SHELBY,0.0339755
EDGAR,0.0333158
DOUGLAS,0.0234193


In [38]:

averaged_weather_data = pd.DataFrame(0, index = range(len(year_month_columns)), columns = weather_df.columns[4:])
for county in weather_df["COUNTY"].drop_duplicates():
    print(len(weather_group_df.get_group(county)[weather_df.columns[4:]] * county_weight_df.loc[county.upper()].iloc[0]))
    temp_df = (weather_group_df.get_group(county)[weather_df.columns[4:]] * county_weight_df.loc[county.upper()].iloc[0]).reset_index(drop = True)
    print(temp_df)
    averaged_weather_data = averaged_weather_data.add(temp_df, fill_value = 0)

370
     TMP_HIGH   TMP_AVG   TMP_LOW   DP_HIGH    DP_AVG    DP_LOW  HUM_HIGH  \
0    3.278095  2.314335  0.590057  2.097981  1.947188  2.097981  6.556190   
1    2.556914  2.294666  2.097981  2.097981  2.097981  2.097981  6.293942   
2    4.195961  3.235948  2.097981  2.097981  2.097981  2.097981  6.293942   
3    5.900571  3.683607  1.901295  3.278095  2.117406  1.442362  6.556190   
4    5.638323  4.278443  2.688038  2.097981  2.097981  2.097981  4.589333   
5    6.425066  4.899659  3.409219  2.097981  2.097981  2.097981  3.081409   
6    6.097257  5.183620  3.933714  2.097981  2.097981  2.097981  2.294666   
7    5.966133  4.650665  2.884724  4.589333  2.104325  2.097981  5.703885   
8    5.900571  4.617743  3.146971  2.097981  2.097981  2.097981  3.540343   
9    4.917142  3.677811  2.097981  2.097981  2.097981  2.097981  6.556190   
10   4.130400  2.624904  2.032419  2.097981  2.095553  1.442362  6.556190   
11   3.015847  2.360228  2.097981  2.491352  2.097981  2.097981  6.55619

     TMP_HIGH   TMP_AVG   TMP_LOW   DP_HIGH    DP_AVG    DP_LOW  HUM_HIGH  \
0    2.812375  1.281938 -1.100495  2.567821  1.078800 -1.283910  6.113859   
1    2.262128  1.379985 -0.427970  2.139851  1.159450 -0.672525  6.113859   
2    5.135642  2.441599  0.427970  3.423761  1.881491 -0.244554  6.113859   
3    5.502473  3.201624  1.528465  3.790593  2.327342  0.733663  6.113859   
4    5.257919  3.770871  2.017574  4.096286  3.094402  1.589603  6.113859   
5    5.563612  4.212449  2.628959  4.524256  3.596987  2.139851  6.113859   
6    5.685889  4.591311  3.301484  4.829949  4.068675  2.934652  6.113859   
7    5.135642  4.106147  2.567821  4.401979  3.496733  2.262128  6.113859   
8    5.319058  3.961781  2.262128  4.401979  3.493052  1.834158  6.113859   
9    4.585394  3.165401  1.711881  4.035147  2.731515  1.589603  6.113859   
10   3.729454  2.052219  0.183416  3.423761  1.595717 -0.183416  6.113859   
11   2.628959  1.711881 -0.122277  2.323267  1.408160 -0.489109  6.113859   

     TMP_HIGH   TMP_AVG   TMP_LOW   DP_HIGH    DP_AVG    DP_LOW  HUM_HIGH  \
0    1.036915  0.482030  0.000000  0.781237  0.311579 -0.241473  1.420431   
1    0.937485  0.499688  0.000000  0.894872  0.421056 -0.142043  1.420431   
2    1.193162  0.678141  0.284086  0.795442  0.476990  0.099430  1.420431   
3    1.278388  0.851312  0.454538  0.923280  0.615520  0.269882  1.377819   
4    1.292593  0.969101  0.610786  1.022711  0.808271  0.454538  1.377819   
5    1.349410  1.106043  0.809646  1.079528  0.944587  0.653398  1.321001   
6    1.448840  1.164754  0.909076  1.107937  1.002092  0.838055  1.335206   
7    1.363614  1.048828  0.653398  1.051119  0.870587  0.511355  1.292593   
8    1.292593  1.048278  0.624990  1.065324  0.888243  0.511355  1.363614   
9    1.164754  0.844469  0.525560  1.036915  0.697844  0.440334  1.363614   
10   1.065324  0.606524  0.184656  0.852259  0.479632  0.028409  1.321001   
11   0.752829  0.492111  0.170452  0.710216  0.383517  0.042613  1.377819   

In [39]:
averaged_weather_data


,TMP_HIGH,TMP_AVG,TMP_LOW,DP_HIGH,DP_AVG,DP_LOW,HUM_HIGH,HUM_AVG,HUM_LOW,SEALVL_HIGH,...,VIS_HIGH,VIS_AVG,VIS_LOW,WIND_HIGH,WIND_AVG,PRECIP,NUMDAY_RAIN,NUM_DAYS_SNOW,NUM_DAYS_FOG,NUM_DAYS_THNDRSTRM
0,55.953658,27.617217,-2.980925,42.880246,19.992262,-8.511984,99.026800,75.092035,32.839506,30.674052,...,18.199743,9.979591,0.186718,29.600197,11.077783,0.479003,2.544772,7.584738,9.707042,0.000000
1,53.838886,28.731112,0.353638,46.537659,23.958440,-4.555208,99.510367,82.286915,39.353377,30.373695,...,16.507919,6.647287,0.109089,24.679482,9.622850,1.722515,5.510271,9.231277,15.203328,0.658457
2,77.668458,41.937001,13.926317,51.205991,29.862688,2.748564,99.138166,65.295138,22.250432,30.590270,...,18.111402,10.209948,0.634101,36.374250,12.349509,15.116988,5.934169,3.868506,9.890731,1.055313
3,85.651529,52.949283,25.351909,60.274139,37.049373,11.479874,99.558871,58.069374,18.204688,30.348293,...,19.391184,11.580387,0.474198,28.918771,10.770362,17.947352,9.195066,1.960945,7.611098,2.058521
4,85.415668,62.338688,36.602102,61.467698,47.555928,27.795232,95.246992,62.031159,21.005582,30.474858,...,19.073952,9.481744,0.280723,33.072546,8.983943,15.212715,9.610446,0.327268,14.161837,4.663441
5,93.180721,71.263159,47.746898,67.021368,55.502022,33.450048,92.041052,62.081519,22.510682,30.306508,...,19.238590,9.691250,0.950015,25.666101,8.317023,10.598875,8.135348,0.000000,12.649299,4.995009
6,93.651100,75.325134,57.224505,71.740380,60.440695,41.882198,91.024753,65.247136,24.452377,30.263593,...,19.073952,9.930980,0.360831,30.924584,7.668664,11.201697,9.899297,0.000000,12.561325,6.286502
7,88.032940,68.355756,43.240501,70.470204,53.535626,34.387899,97.288101,63.037696,24.573892,30.342929,...,18.782894,10.929916,0.123844,23.558382,6.839216,1.162475,6.987723,0.000000,11.848591,1.872076
8,86.350515,66.418060,39.848014,66.092621,53.110278,30.557896,92.975847,67.471453,23.054672,30.374320,...,18.770850,10.109719,0.061922,31.473925,7.953490,6.651649,12.019534,0.109089,13.926898,5.663013
9,74.910045,52.914618,29.721084,61.366736,42.196248,24.559025,98.802348,70.308474,29.572204,30.439866,...,19.150249,9.532099,0.103207,25.729013,7.975400,10.803140,9.754228,0.327268,16.250392,1.152677


In [44]:
averaged_weather_data = pd.concat([year_month_columns, averaged_weather_data], axis=1, join_axes=[averaged_weather_data.index])
averaged_weather_data

,YEAR,MONTH,YEAR,MONTH,TMP_HIGH,TMP_AVG,TMP_LOW,DP_HIGH,DP_AVG,DP_LOW,...,VIS_HIGH,VIS_AVG,VIS_LOW,WIND_HIGH,WIND_AVG,PRECIP,NUMDAY_RAIN,NUM_DAYS_SNOW,NUM_DAYS_FOG,NUM_DAYS_THNDRSTRM
0,1986,JAN,1986,JAN,55.953658,27.617217,-2.980925,42.880246,19.992262,-8.511984,...,18.199743,9.979591,0.186718,29.600197,11.077783,0.479003,2.544772,7.584738,9.707042,0.000000
1,1986,FEB,1986,FEB,53.838886,28.731112,0.353638,46.537659,23.958440,-4.555208,...,16.507919,6.647287,0.109089,24.679482,9.622850,1.722515,5.510271,9.231277,15.203328,0.658457
2,1986,MAR,1986,MAR,77.668458,41.937001,13.926317,51.205991,29.862688,2.748564,...,18.111402,10.209948,0.634101,36.374250,12.349509,15.116988,5.934169,3.868506,9.890731,1.055313
3,1986,ARP,1986,ARP,85.651529,52.949283,25.351909,60.274139,37.049373,11.479874,...,19.391184,11.580387,0.474198,28.918771,10.770362,17.947352,9.195066,1.960945,7.611098,2.058521
4,1986,MAY,1986,MAY,85.415668,62.338688,36.602102,61.467698,47.555928,27.795232,...,19.073952,9.481744,0.280723,33.072546,8.983943,15.212715,9.610446,0.327268,14.161837,4.663441
5,1986,JUN,1986,JUN,93.180721,71.263159,47.746898,67.021368,55.502022,33.450048,...,19.238590,9.691250,0.950015,25.666101,8.317023,10.598875,8.135348,0.000000,12.649299,4.995009
6,1986,JUL,1986,JUL,93.651100,75.325134,57.224505,71.740380,60.440695,41.882198,...,19.073952,9.930980,0.360831,30.924584,7.668664,11.201697,9.899297,0.000000,12.561325,6.286502
7,1986,AUG,1986,AUG,88.032940,68.355756,43.240501,70.470204,53.535626,34.387899,...,18.782894,10.929916,0.123844,23.558382,6.839216,1.162475,6.987723,0.000000,11.848591,1.872076
8,1986,SEP,1986,SEP,86.350515,66.418060,39.848014,66.092621,53.110278,30.557896,...,18.770850,10.109719,0.061922,31.473925,7.953490,6.651649,12.019534,0.109089,13.926898,5.663013
9,1986,OCT,1986,OCT,74.910045,52.914618,29.721084,61.366736,42.196248,24.559025,...,19.150249,9.532099,0.103207,25.729013,7.975400,10.803140,9.754228,0.327268,16.250392,1.152677


In [42]:
averaged_weather_data


,YEAR,MONTH,TMP_HIGH,TMP_AVG,TMP_LOW,DP_HIGH,DP_AVG,DP_LOW,HUM_HIGH,HUM_AVG,...,VIS_HIGH,VIS_AVG,VIS_LOW,WIND_HIGH,WIND_AVG,PRECIP,NUMDAY_RAIN,NUM_DAYS_SNOW,NUM_DAYS_FOG,NUM_DAYS_THNDRSTRM
0,1986,JAN,55.953658,27.617217,-2.980925,42.880246,19.992262,-8.511984,99.026800,75.092035,...,18.199743,9.979591,0.186718,29.600197,11.077783,0.479003,2.544772,7.584738,9.707042,0.000000
1,1986,FEB,53.838886,28.731112,0.353638,46.537659,23.958440,-4.555208,99.510367,82.286915,...,16.507919,6.647287,0.109089,24.679482,9.622850,1.722515,5.510271,9.231277,15.203328,0.658457
2,1986,MAR,77.668458,41.937001,13.926317,51.205991,29.862688,2.748564,99.138166,65.295138,...,18.111402,10.209948,0.634101,36.374250,12.349509,15.116988,5.934169,3.868506,9.890731,1.055313
3,1986,ARP,85.651529,52.949283,25.351909,60.274139,37.049373,11.479874,99.558871,58.069374,...,19.391184,11.580387,0.474198,28.918771,10.770362,17.947352,9.195066,1.960945,7.611098,2.058521
4,1986,MAY,85.415668,62.338688,36.602102,61.467698,47.555928,27.795232,95.246992,62.031159,...,19.073952,9.481744,0.280723,33.072546,8.983943,15.212715,9.610446,0.327268,14.161837,4.663441
5,1986,JUN,93.180721,71.263159,47.746898,67.021368,55.502022,33.450048,92.041052,62.081519,...,19.238590,9.691250,0.950015,25.666101,8.317023,10.598875,8.135348,0.000000,12.649299,4.995009
6,1986,JUL,93.651100,75.325134,57.224505,71.740380,60.440695,41.882198,91.024753,65.247136,...,19.073952,9.930980,0.360831,30.924584,7.668664,11.201697,9.899297,0.000000,12.561325,6.286502
7,1986,AUG,88.032940,68.355756,43.240501,70.470204,53.535626,34.387899,97.288101,63.037696,...,18.782894,10.929916,0.123844,23.558382,6.839216,1.162475,6.987723,0.000000,11.848591,1.872076
8,1986,SEP,86.350515,66.418060,39.848014,66.092621,53.110278,30.557896,92.975847,67.471453,...,18.770850,10.109719,0.061922,31.473925,7.953490,6.651649,12.019534,0.109089,13.926898,5.663013
9,1986,OCT,74.910045,52.914618,29.721084,61.366736,42.196248,24.559025,98.802348,70.308474,...,19.150249,9.532099,0.103207,25.729013,7.975400,10.803140,9.754228,0.327268,16.250392,1.152677
